## Dağıtım Ağı Modelleme

<img src="image3.png" alt="Drawing" style="width: 600px;"/>

### Problem tanımı

Uluslararası bir üretim şirketinin Lojistik Ağı Yöneticisi olarak, taşıma maliyetlerinde son zamanlarda görülen artışı ve gelecekteki talep tahminlerini göz önünde bulundurarak şirketin lojistik ağını önümüzdeki 5 yıl için yeniden tasarlamak istiyorsunuz.

#### Ağ yapısı
* Brezilya, ABD, Hindistan, Japonya, Almanya olmak üzere 5 ülkede üretim ve satış yapıyoruz
* Her ülkeden 2 tip üretim tesisi var: düşük ve yüksek kapasite
* Taşıma maliyetleri (USD/konteyner)
* Piyasa talebi (Adet/yıl)

#### Maliyetler
* Sabit maliyetler (makina/ekipman, kira, vb.): $f$
* Üretim değişken maliyetleri (üretim hattı operatörleri, hammadde, vb.): $v_1$
* Taşıma değişken maliyetleri (1 konteynır = 1000 adet): $v_2$

In [ ]:
%pip install pulp
import pandas as pd
from pulp import *

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
sab_maliyet = pd.read_excel('data.xlsx', 'fixed_costs',index_col = 0)
#sab_maliyet.loc['Hindistan','Yuksek'] *=2
sab_maliyet

In [ ]:
var_costs1 = pd.read_excel('data.xlsx', 'manvar_costs', index_col = 0)
var_costs1

In [ ]:
var_costs2 = pd.read_excel('data.xlsx', 'freight_costs', index_col = 0)/1000
var_costs2 *=5
var_costs2

In [ ]:
deg_maliyet = var_costs1 + var_costs2
deg_maliyet

#### Üretim kapasiteleri

In [ ]:
kapasite = pd.read_excel('data.xlsx', 'capacities', index_col = 0)
#kapasite.loc['Hindistan','Yuksek'] *=2
kapasite

#### Piyasa talepleri

In [ ]:
talep = pd.read_excel('data.xlsx', 'demands', index_col = 0)
talep

In [ ]:
lokasyon = ['ABD', 'Almanya', 'Japonya', 'Brezilya', 'Hindistan']
tip = ['Dusuk', 'Yuksek']

### Model

In [ ]:
model = LpProblem('DagitimAgi',LpMinimize)

#### Karar değişkenleri
* $x_{i,j}$: Lokasyon $i$'den lokasyon $j$'ye transfer miktarı ($i=1,2,3,4,5$, $j=1,2,3,4,5$)
* $y_{i,t}$: Lokasyon $i$'de $t$ kapasiteli tesis kurulacaksa 1, yoksa 0 ($i=1,2,3,4,5$, $t=\{\text{'Dusuk','Yuksek'}\}$)

In [ ]:
x = LpVariable.dicts('transfer', [(i,j) for i in lokasyon for j in lokasyon], lowBound=0, upBound=None, cat='Continuous')
y = LpVariable.dicts('kur', [(i,t) for i in lokasyon for t in tip], cat='Binary')

#### Amaç fonksiyonu
Toplam tesis kurma ve dağıtım maliyetlerini minimize edecek bir dağıtım ağı kurmak istiyoruz:
$$\sum_{i=1}^5 \sum_{t\in\{D,Y\}} f_{i,t} y_{i,t} + \sum_{i=1}^5 \sum_{j=1}^5 v_{i,j} x_{i,j}$$

In [ ]:
model += (lpSum([sab_maliyet.loc[i,t] * y[(i,t)] for i in lokasyon for t in tip]) 
          + lpSum([deg_maliyet.loc[i,j] * x[(i,j)] for i in lokasyon for j in lokasyon]))
model

#### Kısıtlar
* Herbir $i$ lokasyonundan çıkan toplam transferin o lokasyondaki kapasiteyi aşmadığından emin olmalıyız: $$\sum_{j=1}^5 x_{i,j} = \text{Kapasite}_i$$
* Herbir $j$ lokasyonuna gelen toplam transferin o lokasyondaki talebi karşıladığından emin olmalıyız: $$\sum_{i=1}^5 x_{i,j} = \text{Talep}_j$$

In [ ]:
for i in lokasyon:
    model += lpSum([x[(i,j)] for j in lokasyon]) <= lpSum([kapasite.loc[i,t]*y[(i,t)] for t in tip])
for j in lokasyon:
    model += lpSum([x[(i,j)] for i in lokasyon]) == talep.loc[j,'Talep']

#### Modelin çözümü

In [ ]:
model.solve()
print('Toplam maliyet = {:,} ($/Month)'.format(int(value(model.objective))))
print("Durum: {}".format(LpStatus[model.status]))

#### Sonuçlar
* Hangi lokasyona hangi kapasitede bir tesis açıyoruz?
* Hangi lokasyonlar arası ne kadar ürün transfer ediyoruz?

In [ ]:
dict_plant = {}
dict_prod = {}
print('Uretim yapacak fabrikalar ve tipleri')
for v in model.variables():
    if ('kur' in v.name) and (v.varValue == 1):
        name = v.name.replace('kur_', '').replace('_', '')
        dict_plant[name] = int(v.varValue)
        #p_name = name
        print(name)#, '=', capacity.loc[p_name,])#, "=", v.varValue)
print('\nUretim/transfer miktarlari')
for v in model.variables():
    if ('transfer' in v.name) and (v.varValue > 0):
        name = v.name.replace('transfer_', '').replace('_', '')
        dict_prod[name] = v.varValue
        print(name, "=", '{:,} adet'.format(int(value(v.varValue))))
#capacities

#### Senaryo A: Üretimi işgücü maliyeti düşük olan bölgeye kaydırma
Hindistan'daki yüksek kapasiteli fabrikanın kapasitesini 2 katına çıkarırsak ne olur? (Sabit maliyetin de 2 katına çıktığını varsayalım.)
* Brazil plant is still producing for the local market and the USA
* India plants produce for all countries except Brazil
* Japan does not produce locally anymore.
* Total Costs = 51,352,000 \$/Month (19.4\% less than Scenario 1)

#### Senaryo B: Konteynır kısıtı nedeniyle artan taşıma maliyetleri
Konteynır kısıtı nedeniyle taşıma maliyetleri 5 katına çıkarsa ne olur?
* Brazil is producing for the local market only
* The USA started to produce for the local market and Japan
* India closed its low capacity factory
* Japan starts to produce for its local market
* Because of their limited production capacity, Japan and the USA still rely on the Indian plant
* Total Costs = 92,981,000 $/Month